# 《安娜卡列尼娜》新编——利用TensorFlow构建LSTM模型

最近看完了LSTM的一些外文资料，主要参考了Colah的blog以及Andrej Karpathy blog的一些关于RNN的材料，准备动手去实现一个LSTM模型。代码的基础框架来自于Udacity上深度学习纳米学位的课程（付费课程）的一个demo，我刚开始看代码的时候真的是一头雾水，很多东西没有理解，后来反复查阅资料，并我重新对代码进行了学习和修改，对步骤进行了进一步的剖析，下面将一步步用TensorFlow来构建LSTM模型进行文本学习并试图去生成新的文本。

关于RNN与LSTM模型本文不做介绍，详情去查阅资料过着去看上面的blog链接，讲的很清楚啦。这篇文章主要是偏向实战，来自己动手构建LSTM模型。

数据集来自于外文版《安娜卡列妮娜》书籍的文本文档（本文后面会提供整个project的git链接）。

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

# 1 数据加载与预处理

In [2]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
print(vocab_to_int)
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

{'N': 0, ' ': 1, '?': 2, 'D': 3, 't': 4, 'c': 5, 'v': 6, '-': 7, '3': 8, 'r': 9, 'V': 10, 'h': 11, '6': 12, '$': 13, 'O': 14, 'U': 15, '2': 16, 'A': 17, 'G': 18, '"': 19, '`': 20, 'T': 21, '7': 22, 'f': 23, ';': 24, ')': 25, 'z': 26, 'j': 27, 'l': 28, 'k': 29, '9': 30, 'K': 31, '1': 32, 'b': 33, 'm': 34, 'i': 35, ':': 36, 'Z': 37, '8': 38, 'J': 39, 'P': 40, 'g': 41, 'C': 42, '!': 43, 'y': 44, "'": 45, 'q': 46, '4': 47, 'I': 48, '/': 49, 'L': 50, ',': 51, 'Y': 52, 'Q': 53, 'W': 54, '*': 55, '@': 56, '&': 57, '\n': 58, 'R': 59, 'u': 60, 'S': 61, '0': 62, 'M': 63, 'X': 64, 'e': 65, 'o': 66, 'p': 67, 'a': 68, 'E': 69, '.': 70, 'd': 71, '_': 72, '(': 73, 'n': 74, 's': 75, '%': 76, 'w': 77, '5': 78, 'x': 79, 'B': 80, 'F': 81, 'H': 82}


In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [4]:
encoded[:100]

array([42, 11, 68, 67,  4, 65,  9,  1, 32, 58, 58, 58, 82, 68, 67, 67, 44,
        1, 23, 68, 34, 35, 28, 35, 65, 75,  1, 68,  9, 65,  1, 68, 28, 28,
        1, 68, 28, 35, 29, 65, 24,  1, 65,  6, 65,  9, 44,  1, 60, 74, 11,
       68, 67, 67, 44,  1, 23, 68, 34, 35, 28, 44,  1, 35, 75,  1, 60, 74,
       11, 68, 67, 67, 44,  1, 35, 74,  1, 35,  4, 75,  1, 66, 77, 74, 58,
       77, 68, 44, 70, 58, 58, 69,  6, 65,  9, 44,  4, 11, 35, 74], dtype=int32)

In [5]:
len(vocab)

83

# 2 分割mini-batch



<img src="assets/sequence_batching@1x.png" width=500px>


完成了前面的数据预处理操作，接下来就是要划分我们的数据集，在这里我们使用mini-batch来进行模型训练，那么我们要如何划分数据集呢？在进行mini-batch划分之前，我们先来了解几个概念。

假如我们目前手里有一个序列1-12，我们接下来以这个序列为例来说明划分mini-batch中的几个概念。首先我们回顾一下，在DNN和CNN中，我们都会将数据分batch输入给神经网络，加入我们有100个样本，如果设置我们的batch_size=10，那么意味着每次我们都会向神经网络输入10个样本进行训练调整参数。同样的，在LSTM中，batch_size意味着每次向网络输入多少个样本，在上图中，当我们设置batch_size=2时，我们会将整个序列划分为6个batch，每个batch中有两个数字。

然而由于RNN中存在着“记忆”，也就是循环。事实上一个循环神经网络能够被看做是多个相同神经网络的叠加，在这个系统中，每一个网络都会传递信息给下一个。上面的图中，我们可以看到整个RNN网络由三个相同的神经网络单元叠加起来的序列。那么在这里就有了第二个概念sequence_length（也叫steps），中文叫序列长度。上图中序列长度是3，可以看到将三个字符作为了一个序列。

有了上面两个概念，我们来规范一下后面的定义。我们定义一个batch中的序列个数为N（batch_size），定义单个序列长度为M（也就是我们的steps）。那么实际上我们每个batch是一个N x M的数组。在这里我们重新定义batch_size为一个N x M的数组，而不是batch中序列的个数。在上图中，当我们设置N=2， M=3时，我们可以得到每个batch的大小为2 x 3 = 6个字符，整个序列可以被分割成12 / 6 = 2个batch。

In [6]:
def get_batches(arr, n_seqs, n_steps):
    '''
    对已有的数组进行mini-batch分割
    
    arr: 待分割的数组
    n_seqs: 一个batch中序列个数
    n_steps: 单个序列包含的字符数
    '''
    
    batch_size = n_seqs * n_steps
    n_batches = int(len(arr) / batch_size)
    # 这里我们仅保留完整的batch，对于不能整出的部分进行舍弃
    arr = arr[:batch_size * n_batches]
    
    # 重塑
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # inputs
        x = arr[:, n:n+n_steps]
        # targets
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

上面的代码定义了一个generator，调用函数会返回一个generator对象，我们可以获取一个batch。

In [7]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [8]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[42 11 68 67  4 65  9  1 32 58]
 [ 1 68 34  1 74 66  4  1 41 66]
 [ 6 35 74 70 58 58 19 52 65 75]
 [74  1 71 60  9 35 74 41  1 11]
 [ 1 35  4  1 35 75 51  1 75 35]
 [ 1 48  4  1 77 68 75 58 66 74]
 [11 65 74  1  5 66 34 65  1 23]
 [24  1 33 60  4  1 74 66 77  1]
 [ 4  1 35 75 74 45  4 70  1 21]
 [ 1 75 68 35 71  1  4 66  1 11]]

y
 [[11 68 67  4 65  9  1 32 58 58]
 [68 34  1 74 66  4  1 41 66 35]
 [35 74 70 58 58 19 52 65 75 51]
 [ 1 71 60  9 35 74 41  1 11 35]
 [35  4  1 35 75 51  1 75 35  9]
 [48  4  1 77 68 75 58 66 74 28]
 [65 74  1  5 66 34 65  1 23 66]
 [ 1 33 60  4  1 74 66 77  1 75]
 [ 1 35 75 74 45  4 70  1 21 11]
 [75 68 35 71  1  4 66  1 11 65]]


# 3 模型构建
模型构建部分主要包括了输入层，LSTM层，输出层，loss，optimizer等部分的构建，我们将一块一块来进行实现。

## 3.1 输入层

In [9]:
def build_inputs(num_seqs, num_steps):
    '''
    构建输入层
    
    num_seqs: 每个batch中的序列个数
    num_steps: 每个序列包含的字符数
    '''
    inputs = tf.placeholder(tf.int32, shape=(num_seqs, num_steps), name='inputs')
    targets = tf.placeholder(tf.int32, shape=(num_seqs, num_steps), name='targets')
    
    # 加入keep_prob
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

## 3.2 LSTM层

In [10]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' 
    构建lstm层
        
    keep_prob
    lstm_size: lstm隐层中结点数目
    num_layers: lstm的隐层数目
    batch_size: batch_size

    '''
    # 构建一个基本lstm单元
    #lstm = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
    # 添加dropout
    #drop = tf.nn.rnn_cell.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    # 堆叠
    #cell = tf.nn.rnn_cell.MultiRNNCell([drop for _ in range(num_layers)])
    
    stack_drop=[]
    for i in range(num_layers):
        lstm = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
        drop = tf.nn.rnn_cell.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        stack_drop.append(drop)
        cell = tf.nn.rnn_cell.MultiRNNCell(stack_drop, state_is_tuple = True)
        
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

## 3.3 输出层

In [11]:
def build_output(lstm_output, in_size, out_size):
    ''' 
    构造输出层
        
    lstm_output: lstm层的输出结果
    in_size: lstm输出层重塑后的size
    out_size: softmax层的size
    
    '''

    # 将lstm的输出按照列concate，例如[[1,2,3],[7,8,9]],
    # tf.concat的结果是[1,2,3,7,8,9]
    #seq_output = tf.concat(1, lstm_output) # tf.concat(concat_dim, values)
    seq_output = tf.concat(lstm_output, 1)

    # reshape
    x = tf.reshape(seq_output, [-1, in_size])
    
    # 将lstm层与softmax层全连接
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal([in_size, out_size], stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # 计算logits
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # softmax层返回概率分布
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

## 3.4 训练误差计算

In [12]:
def build_loss(logits, targets, lstm_size, num_classes):
    '''
    根据logits和targets计算损失
    
    logits: 全连接层的输出结果（不经过softmax）
    targets: targets
    lstm_size
    num_classes: vocab_size
        
    '''
    
    # One-hot编码
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    
    return loss

## 3.5 Optimizer
我们知道RNN会遇到梯度爆炸（gradients exploding）和梯度弥散（gradients disappearing)的问题。LSTM解决了梯度弥散的问题，但是gradient仍然可能会爆炸，因此我们采用gradient clippling的方式来防止梯度爆炸。即通过设置一个阈值，当gradients超过这个阈值时，就将它重置为阈值大小，这就保证了梯度不会变得很大。

In [13]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' 
    构造Optimizer
   
    loss: 损失
    learning_rate: 学习率
    
    '''
    
    # 使用clipping gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

## 3.6 模型组合
使用tf.nn.dynamic_run来运行RNN序列

In [14]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # 如果sampling是True，则采用SGD
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # 输入层
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # LSTM层
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        # 对输入进行one-hot编码
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # 运行RNN
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # 预测结果
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss 和 optimizer (with gradient clipping)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

# 4 模型训练

### 参数设置
在模型训练之前，我们首先初始化一些参数，我们的参数主要有：

- num_seqs: 单个batch中序列的个数
- num_steps: 单个序列中字符数目
- lstm_size: 隐层结点个数
- num_layers: LSTM层个数
- learning_rate: 学习率
- keep_prob: dropout层中保留结点比例

In [15]:
batch_size = 100         # Sequences per batch
num_steps = 100          # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001    # Learning rate
keep_prob = 0.5         # Dropout keep probability

In [ ]:
epochs = 20
# 每n轮进行一次变量保存
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    counter = 0
    for e in range(epochs):
        print(e)
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            # control the print lines
            if counter % 100 == 0:
                print('轮数: {}/{}... '.format(e+1, epochs),
                      '训练步数: {}... '.format(counter),
                      '训练误差: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end-start)))

            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

0
轮数: 1/20...  训练步数: 100...  训练误差: 3.0772...  8.1298 sec/batch
1
轮数: 2/20...  训练步数: 200...  训练误差: 2.4709...  7.9561 sec/batch
轮数: 2/20...  训练步数: 300...  训练误差: 2.2659...  8.7499 sec/batch
2
轮数: 3/20...  训练步数: 400...  训练误差: 2.1057...  7.9027 sec/batch
轮数: 3/20...  训练步数: 500...  训练误差: 1.9686...  8.7879 sec/batch
3
轮数: 4/20...  训练步数: 600...  训练误差: 1.8405...  8.3461 sec/batch
轮数: 4/20...  训练步数: 700...  训练误差: 1.8028...  7.7966 sec/batch
4
轮数: 5/20...  训练步数: 800...  训练误差: 1.7376...  8.0579 sec/batch
轮数: 5/20...  训练步数: 900...  训练误差: 1.6995...  8.0308 sec/batch
5
轮数: 6/20...  训练步数: 1000...  训练误差: 1.6258...  10.2623 sec/batch
轮数: 6/20...  训练步数: 1100...  训练误差: 1.6132...  8.6325 sec/batch
6
轮数: 7/20...  训练步数: 1200...  训练误差: 1.5342...  8.5939 sec/batch
轮数: 7/20...  训练步数: 1300...  训练误差: 1.5090...  7.9916 sec/ba

In [16]:
# 查看checkpoints
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i1800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1800_l512.ckpt"

# 5 文本生成
现在我们可以基于我们的训练参数进行文本的生成。当我们输入一个字符时，LSTM会预测下一个字符，我们再将新的字符进行输入，这样能不断的循环下去生成本文。

为了减少噪音，每次的预测值我会选择最可能的前5个进行随机选择，比如输入h，预测结果概率最大的前五个为[o,e,i,u,b]，我们将随机从这五个中挑选一个作为新的字符，让过程加入随机因素会减少一些噪音的生成。

In [17]:
def pick_top_n(preds, vocab_size, top_n=5):
    """
    从预测结果中选取前top_n个最可能的字符
    
    preds: 预测结果
    vocab_size
    top_n
    """
    p = np.squeeze(preds)
    # 将除了top_n个预测值的位置都置为0
    p[np.argsort(p)[:-top_n]] = 0
    # 归一化概率
    p = p / np.sum(p)
    # 随机选取一个字符
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [18]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    """
    生成新文本
    
    checkpoint: 某一轮迭代的参数文件
    n_sample: 新闻本的字符长度
    lstm_size: 隐层结点数
    vocab_size
    prime: 起始文本
    """
    # 将输入的单词转换为单个字符组成的list
    samples = [c for c in prime]
    # sampling=True意味着batch的size=1 x 1
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        # 加载模型参数，恢复训练
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            # 输入单个字符
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        # 添加字符到samples中
        samples.append(int_to_vocab[c])
        
        # 不断生成字符，直到达到指定数目
        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [19]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints/i1800_l512.ckpt'

In [20]:
# 选用最终的训练参数作为输入进行文本生成
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="The")
print(samp)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

INFO:tensorflow:Restoring parameters from checkpoints/i1800_l512.ckpt
The5*?,bmb DjotHm*?of5wmi*bmb * 3?vm* m5w-m 5* tm* m 53bms*bb3D?mi3 53wm*msDb3 3D?tm*?om5wfi*bm 5wms-wewbbm3?m 5wm9*e3b3?vm53bmi39wtm*jjm5wmeDnjom?D m 5wfs-3?ewbbmi3 5mb5wmeDnjom?D mbD?m 5wmsw-bD*bmb 3jjmbw*-m3?m5w-m Df 53bmi39w,bm*?ci5w-wtm 5wcm5*oml-Dnv5 m538mD99m3?mbw?3?vmD?m5w-7ffH)Dnmu?Dibm5*$wtm*?omb5wm5*om 53btmln m 53?ubm3?mbne5tm*?om5wmb*cm Df 53?um Dmb5wm5*bm -Dnlwotmb5wm5*bm Djom 5wm83v5 m* mD?ewm*buwom Dmlwfb DjotHmb5wmb*3otm*?om5wme*- n-womlcm5387ffH65wmeDn? -cmbD8w 53?vKmr5* ,bm 5wme*?mD9m*mj3  jwme5*jjm*lDn m538tfiw-wtmi5* m*jjmcDnm*?biw-Kmr5* mi*cm 5wm8*?m*lDn m5w-m3bm?wewbb*-ctf 5w?tm*?om 5* ,bm*mj3  jwm8D?o3?vm 38wm*?om8wmiD-um3 tm*?omTme*-wm* f5w-wm*?omi5* m39mcDn-meDn- 3?vm -Dnv5 tm*?om 5wm83b -w? tmi*bm*?cm8

In [21]:
checkpoint = 'checkpoints/i200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i200_l512.ckpt
FarHfHfr5* m *m 5wm 3-m 5mbD-wbtm 5D-m wm 5*-m* m3 m5wm w-mD?wm5wmi5wm 5m* 3om5w m*?bmi*jo3?v7fHfrDn m*?mDDm5wbw-w-m3bomi53 m* m 5w-wmb* D m*?om5w-wbm5w mD?m53-m *bmD 5w-m*9m 5wmi53bm5D mDn wm * mD?m 5wm *- wm 5D-m 53bmi5wm D?wmbw-m* 3?m 3b5wmiw?bm*?owmb5*? m53 m53 m5w-wmbwbm*?vm * mDn m*?mDDbm*?wm*?m Dm 5D?f D-mDnom Dm 53-m 5*?miD-wbmbDbw-mDn?m 5mb3bwbmD?ow?ow D mD-wbm*?m*o w mD?vm 5wbmbwm53-wmbw m5wmiDbmb38wm *?m 5wmbD-wbm53 mD?bwbm53?mb3j wm5wm 53-m*? 5w? m*bm 5*bom 5Dm83? wmbw-wm w-m5*bm Dn mi3 5m D8m Dm553?vmb38wm Dm wmbwb3?vm5wmb3?m D-fw? w-tm53?vm Dm 5w-mbDbm*jm 5w-mi* 5m*bwb* m*?vm5wmiD-wmb3bwm*9mD9m3 Dm wm*-w m53-wofD-mDbm 3?m5*bwmb3?ow Dm9D m D-w-*?bm D-m 5wm D-wmi5w-m wbm ww m*?vw?m5D-m*b D wmiw-m*bwm 5* m 5wmbD-m5D fwbwm D m 5wm5*? m*? m* m5wmbD?ofbD-3?mD?m D-fDnomb5* m*?omi5*mbmD?bm 5wmbD?wmi5*? m 5w m5wbm wm5wmbD w-m 5wm 5*bwbtm*?omi5*-mb3? m5D?mbD wmiD mD?m D m5* m53bmbDn?vm 5wm5Dbm w-m*?om*?om *?vm*? m

In [22]:
checkpoint = 'checkpoints/i1000_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i1000_l512.ckpt
Farm5wm5*om 5* mb5wmi*bm Dmbwj9m 5wm8D8w?m 5-wwmb*8wm 5w3-mi*bm 5wmi5D8wbm*?om5*$wmb*cm?D flw9Djwf 5* m5wm5*om*?-mb5wmiw- m*?omb*3om*?om*jjw*ocm Dm 5wmbDm 5* m*bm53bmbns9D-m5*of 5Dnv5 mD$w-m*lDn m5*om 5wmlD 5f 5w3-m5*ss*?w-bmD?m5w-m 5-3wbtm*?om*?ofb*3om*bm 5-Dnv5mb5wm5*om*jjmbD8w 53?vf538m*?om5*omlww?mln m5w-m5wjoml-w*owomlnv5woflww?m5w* wom39m 5w-mbD8wmD9m 5* m5*$wm Dm 5wfs-w?bbmD9m5w*- m -3?om*?omb*i3*vm Dm Dm*?om*?om53bmlDnjomD9f53bmiD-obtm 5wmbw8wj tmi5* m5wmi*bmb DssDowom3?m5w-fb w*um*?om DDum*lDn m3?m5w-m*jjm5Difi3 5m 5w3-mi*bm*ms-D$3jjcmb*cm*?om5*omln m*mjD$wmD-m5*$wmlcm53bm9* w-m39m*bm 5wmbw8w  3D?m 5* m53bmb53jjmi39wfb *s3?vm 5Dnv5 tm*ms-Dnb3?vm 38sjcm 5wmbD8w 53?vtm*?omb*3om 5* m5wmi*bm*?-wom53bfe-nwomD$m53bm9D-8D?wtmi5Dmi*bm*?om Dm5*$wmlww?m* m3 7mhn mi*bm*jjm 5wfiw-wm D8wtm*msjDiw-mi*bm 5wf8*? wm 5* m5wm5*om*mbw? fD9m*mbn-?wbm*9m*?m ww-womD9mb 3jj3?vmD9m 5wmsjw*bwmD9 w-f5w*- 7mp5wmeD?bwo-wom*?om 5wme*jjw-

In [24]:
checkpoint = 'checkpoints/i1800_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i1800_l512.ckpt
Farf5wme*-wom Dm 5wmbD8wm*-8*?m*?om*mb83jwmD9m53bmbw?bwtmln mbw? j*?of 5* m 5wcm -Dwbm* m 5wme*8sjwmD9m*mb3b w-mD9m5nbl*?o7ffH/5tm3 ,bm*m8*?m 5* m3 ,jjm8*?m*?om*mb3owmD9m8w* D-7m653bmi*bmb5wmo3ofb -neum3 7ffp ws*?mX-u*ocw$3 e5,bmbD? w-wom3? Dm538bwj9Um*?om 5wm8D8w? mi*bfb DDomb 3jjm8wmD9m3 7mp5wmb*im wmbww m5w-m5w*- 7mpw-cD(5*m 5Dnv5 tmi5DfiDnjomlwm*mv-DD8mD9m3? w-wb m*bm 5wmbDjw8*?mD9m 5-3eum 5w8m5Dim5wmb*if 5Dnv5 mD9m5w-m5*?obtm 5* m 53bmi39wm*m83?om 5wmbw-$3?wtmi5Dmi*bm DfeD8sj*cD?m53bmbD?m*?om 5w?mi* mb3  jwm Dmbww8m 5wmbDn?om*?om 5wm9-D8m 5wfeD-ow-7m&wmiw? mD?m*mjw m 5* m3bm 5wms-w*bw-bm*b we 3$3jcm3?owwomD9m3 fi*bmbDjwmbD8wm*bmbDD?m*?omiw- m3?m 5wmsw*b*? btm*?om 5wme53?ew-bm*?of -3?uwomD?m 5wme5*?vwm 5wme*-wtmb*c3?vm*?om5w*- tm*?omiw? mnsm*?omb *--wom5w-f5nb *-otmb5wmeDnjom?D m Dmo3?om 5wms-* w-mi5w?m5wmowowom 5* m*jjm5wf5*bmbww?m 5* m5wm5*om Dmb*3otm*?omi*bm?D m Dmlwm DmvDm Dm538m*mo3?vm538bwj9f*?om 5wmeD?$w-b